## Querying a Milvus index

Simple example on how to query content from a Milvus Vector Store.

### Requirements

- A **Milvus** vector database, set up according to [these instructions](../../../vector-databases/milvus/README.md).

- Connection credentials to Milvus must be available as environment variables:

  - `MILVUS_USERNAME`

  - `MILVUS_PASSWORD`

- Update the **MILVUS_HOST**, **MILVUS_PORT**, and **MILVUS_COLLECTION** in this notebook to match your deployment settings.

### Needed packages and imports

In [1]:
!pip install -q einops==0.7.0 langchain==0.1.9 pymilvus==2.3.6 sentence-transformers==2.4.0

In [2]:
import os
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Milvus

### Base parameters, the Milvus connection info

In [3]:
# Replace values according to your Milvus deployment
MILVUS_HOST = "milvus-service"
# MILVUS_HOST = "milvus-service.<your-namespace>.svc.cluster.local"
MILVUS_PORT = 19530
MILVUS_USERNAME = os.getenv('MILVUS_USERNAME')
MILVUS_PASSWORD = os.getenv('MILVUS_PASSWORD')
MILVUS_COLLECTION = "demo_collection"

### Initialize the connection

In [4]:
# If you want to use a GPU, you can add the 'device': 'cuda' argument and provided you have used GPU Accelerator on your Workbench.
# model_kwargs = {'device': 'cuda'}
model_kwargs = {}

embeddings = HuggingFaceEmbeddings(
    model_kwargs=model_kwargs,
    show_progress=True
)

store = Milvus(
    embedding_function=embeddings,
    connection_args={"host": MILVUS_HOST, "port": MILVUS_PORT, "user": MILVUS_USERNAME, "password": MILVUS_PASSWORD},
    collection_name=MILVUS_COLLECTION,
    metadata_field="metadata",
    text_field="page_content",
    drop_old=False
    )

/opt/app-root/lib64/python3.9/site-packages/networkx/utils/backends.py:135: RuntimeWarning: networkx backend defined more than once: nx-loopback
  backends.update(_get_backends("networkx.backends"))
2025-05-31 03:00:38.482764: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-31 03:00:39.402981: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/opt/app-root/lib64/python3.9/site-packages/pymilvus/client/__init__.py:6: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import Distribution

### Make a query to the index to verify sources

In [5]:
query="How can I create a Data Science Project?"
results = store.similarity_search_with_score(query, k=4, return_metadata=True)
for result in results:
    print(result[0].metadata['source'])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

https://access.redhat.com/documentation/en-us/red_hat_openshift_ai_self-managed/2.20/html-single/openshift_ai_tutorial_-_fraud_detection_example/index
https://access.redhat.com/documentation/en-us/red_hat_openshift_ai_self-managed/2.20/html-single/working_on_data_science_projects/index
https://access.redhat.com/documentation/en-us/red_hat_openshift_ai_self-managed/2.20/html-single/working_on_data_science_projects/index
https://access.redhat.com/documentation/en-us/red_hat_openshift_ai_self-managed/2.20/html-single/getting_started_with_red_hat_openshift_ai_self-managed/index


### Work with a retriever

In [6]:
retriever = store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [7]:
docs = retriever.get_relevant_documents(query)
docs

/opt/app-root/lib64/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[Document(page_content='.\nProcedure\n1\n. \nOn the navigation menu, select \nData science projects\n. This page lists any existing projects that\nyou have access to. From this page, you can select an existing project (if any) or create a new\none.\nNOTE\nCHAPTER 2. SETTING UP A PROJECT AND STORAGE\n5', metadata={'source': 'https://access.redhat.com/documentation/en-us/red_hat_openshift_ai_self-managed/2.20/html-single/openshift_ai_tutorial_-_fraud_detection_example/index', 'page': 8}),
 Document(page_content='. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 